In [1]:
import os
os.environ['JAVA_HOME'] = 'C:/Program Files/Java/jdk1.8.0_291'

In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf())

In [3]:
import numpy as np

# 定义距离函数

In [4]:
def l2_dist(u, v):
    return np.abs((u*u-v*v)).sum()

In [5]:
def l1_dist(u, v):
    return np.abs((u-v)).sum()

# 聚类时将数据点指派给簇的方法

In [6]:
# return (cluster, point, cost)
def assign_to_cluster(point, centroids, dist_fn):
    min_dist = float('inf')
    cluster = None
    for i, centroid in enumerate(centroids.value):
        # TODO: YOUR CODE HERE
        dist = dist_fn(point, centroid)
        if dist < min_dist:
            min_dist = dist
            cluster = i
    return cluster, point, min_dist

# 文件路径

In [7]:
data_fname = 'q1/data/data.txt'
random_initial_centroids_fname = 'q1/data/c1.txt'
far_initial_centroids_fname = 'q1/data/c2.txt'

# 加载数据点

In [8]:
data = sc.textFile(data_fname)
data = data.map(lambda line: np.array([float(value) for value in line.split()]))

# 加载初始质心

In [9]:
centroids = np.loadtxt(far_initial_centroids_fname)
centroids = sc.broadcast(centroids)

In [10]:
dist_fn = l2_dist
MAX_ITER = 20
cost_seq = []
for i in range(MAX_ITER):
    assignment = data.map(lambda point: assign_to_cluster(point, centroids, dist_fn))
    
    cost = assignment.map(lambda pc:pc[2]).reduce(lambda a,b:a+b)
    print(f'iter {i}: cost is {cost}')
    cost_seq.append(cost)

    assignment = assignment.map(lambda pc:(pc[0],(pc[1],pc[2])))
    assignment = assignment.reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1]))
    assignment = assignment.mapValues(lambda pc:pc[0]/pc[1])
    new_centroids = assignment.map(lambda pc:pc[1]).collect()
    
    centroids = sc.broadcast(new_centroids)


iter 0: cost is 986043280.0149491
iter 1: cost is 2252657603.448908
iter 2: cost is 2252657603.565752
iter 3: cost is 2252657603.565752
iter 4: cost is 2252657603.565752
iter 5: cost is 2252657603.565752
iter 6: cost is 2252657603.565752
iter 7: cost is 2252657603.565752
iter 8: cost is 2252657603.565752
iter 9: cost is 2252657603.565752
iter 10: cost is 2252657603.565752
iter 11: cost is 2252657603.565752
iter 12: cost is 2252657603.565752
iter 13: cost is 2252657603.565752
iter 14: cost is 2252657603.565752
iter 15: cost is 2252657603.565752
iter 16: cost is 2252657603.565752
iter 17: cost is 2252657603.565752
iter 18: cost is 2252657603.565752
iter 19: cost is 2252657603.565752


In [ ]:
dist_fn = l1_dist
MAX_ITER = 20
cost_seq = []
for i in range(MAX_ITER):
    assignment = data.map(lambda point: assign_to_cluster(point, centroids, dist_fn))
    
    cost = assignment.map(lambda pc:pc[2     ]).reduce(lambda a,b:a+b)
    print(f'iter {i}: cost is {cost}')
    cost_seq.append(cost)

    assignment = assignment.map(lambda pc:(pc[0],(pc[1],pc[2])))
    assignment = assignment.reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1]))
    assignment = assignment.mapValues(lambda pc:pc[0]/pc[1])
    new_centroids = assignment.map(lambda pc:pc[1]).collect()
    
    centroids = sc.broadcast(new_centroids)